In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [82]:
#!pip install matplotlib==2.2.0
#!pip install sklearn-genetic

ERROR: Could not find a version that satisfies the requirement evolutionary-search
ERROR: No matching distribution found for evolutionary-search


In [2]:
#Read files
vtodc_file = r'C:\D_Drive\Data\ASU Project.Vendor Fill Rate data 2020.2021 - adjusted.xlsx'


excel_file = pd.ExcelFile(vtodc_file)
sheets = excel_file.sheet_names
print('sheets in file: ',sheets)
vtodc_df=pd.DataFrame()
# loop through sheets inside an Excel file
for sheet in sheets: 
    if sheet!='Summary':
        #print(sheet)
        df = excel_file.parse(sheet_name = sheet)
        vtodc_df = vtodc_df.append(df)

excel_file.close()
print(vtodc_df.shape)
print(vtodc_df.head())


sheets in file:  ['Summary', 'Vendor 1 2020', 'Vendor 2 2020', 'Vendor 3 2020', 'Vendor 4 2020', 'Vendor 5 2020', 'Vendor 6 2020', 'Vendor 1 2021', 'Vendor 2 2021', 'Vendor 3 2021', 'Vendor 4 2021', 'Vendor 5 2021', 'Vendor 6 2021']
(196235, 16)
  BUS_GRP_NM  DIVISION  YEAR  VEND_NUM DST_CNTR  FACILITY  CORP_ITEM_CD  \
0     BAKERY        17  2020         1     WPHX      1721      94984302   
1     BAKERY        19  2020         1     WPOA      5316      94984302   
2     BAKERY        20  2020         1     WDAL      2018      94984302   
3     BAKERY        20  2020         1     WDAL      2018      94984302   
4     BAKERY        20  2020         1     WDAL      2018      94984302   

   DAYS_LATE  PO_NUM  ORDERED  AMT_RECV  SHORTED  FILLRATE     PO_ORD  \
0          3  981010      937       937        0     100.0 2019-12-13   
1          0  674833      500       500        0     100.0 2019-12-02   
2          0  514961      668       668        0     100.0 2019-12-12   
3          

In [3]:
vtodc_df.isnull().sum().sort_values(ascending=False)

BUS_GRP_NM      0
DIVISION        0
YEAR            0
VEND_NUM        0
DST_CNTR        0
FACILITY        0
CORP_ITEM_CD    0
DAYS_LATE       0
PO_NUM          0
ORDERED         0
AMT_RECV        0
SHORTED         0
FILLRATE        0
PO_ORD          0
PO_DUE          0
DATE_RECV       0
dtype: int64

In [4]:
vtodc_df['FILLRATE'].describe()

count    196235.000000
mean         92.445693
std         236.543128
min           0.000000
25%         100.000000
50%         100.000000
75%         100.000000
max       67600.000000
Name: FILLRATE, dtype: float64

In [5]:
vtodc_df['MONTH'] = vtodc_df['DATE_RECV'].dt.month
vtodc_df['WEEK'] = vtodc_df['DATE_RECV'].dt.week
vtodc_df['DAY'] = vtodc_df['DATE_RECV'].dt.day

In [6]:
vtodc_df['MONTH'].value_counts()

10    18166
3     17543
9     17519
4     17471
6     17427
8     16846
11    16398
7     16156
5     15935
12    15002
2     14166
1     13606
Name: MONTH, dtype: int64

In [11]:
vtodc_df['OrderFulfillment']=np.where(vtodc_df['FILLRATE']>=80,1,0)
print(vtodc_df['OrderFulfillment'].value_counts())

1    176179
0     20056
Name: OrderFulfillment, dtype: int64


In [64]:
#Creating final test or hold-out set
test=vtodc_df.copy().sample(5000)
print(test.shape)
print(test['OrderFulfillment'].value_counts())

(5000, 20)
0    2511
1    2489
Name: OrderFulfillment, dtype: int64


In [66]:
#UnderSampling: Since this is an imbalanced dataset, sampling equal number of datapoints from both clases
vtodc_df_1=vtodc_df.copy().query("OrderFulfillment==1")
vtodc_df_0=vtodc_df.copy().query("OrderFulfillment==0")
vtodc_df_1=vtodc_df_1.copy().sample(len(vtodc_df_0))
len(vtodc_df_1)
train=pd.concat([vtodc_df_0,vtodc_df_1], axis=0)
train.shape

(40112, 20)

In [68]:
#Check Target column distribution in train and test data
print(train['OrderFulfillment'].value_counts())
print(test['OrderFulfillment'].value_counts())

0    20056
1    20056
Name: OrderFulfillment, dtype: int64
0    2511
1    2489
Name: OrderFulfillment, dtype: int64


In [69]:
#Data cleaning
train[['CORP_ITEM_CD','DIVISION','VEND_NUM','FACILITY','MONTH','WEEK','DAY']]=train[['CORP_ITEM_CD','DIVISION','VEND_NUM','FACILITY','MONTH','WEEK','DAY']].astype(str)
test[['CORP_ITEM_CD','DIVISION','VEND_NUM','FACILITY','MONTH','WEEK','DAY']]=test[['CORP_ITEM_CD','DIVISION','VEND_NUM','FACILITY','MONTH','WEEK','DAY']].astype(str)

print(test.dtypes)
print(train.dtypes)

BUS_GRP_NM                  object
DIVISION                    object
YEAR                         int64
VEND_NUM                    object
DST_CNTR                    object
FACILITY                    object
CORP_ITEM_CD                object
DAYS_LATE                    int64
PO_NUM                       int64
ORDERED                      int64
AMT_RECV                     int64
SHORTED                      int64
FILLRATE                   float64
PO_ORD              datetime64[ns]
PO_DUE              datetime64[ns]
DATE_RECV           datetime64[ns]
MONTH                       object
WEEK                        object
DAY                         object
OrderFulfillment             int32
dtype: object
BUS_GRP_NM                  object
DIVISION                    object
YEAR                         int64
VEND_NUM                    object
DST_CNTR                    object
FACILITY                    object
CORP_ITEM_CD                object
DAYS_LATE                    int64
PO_NUM

In [70]:
del train['YEAR']
del train['FILLRATE']
del train['PO_NUM']
del train['PO_ORD']
del train['PO_DUE']
del train['DATE_RECV']
del train['SHORTED']
del train['AMT_RECV']
print(train.dtypes)
del test['YEAR']
del test['FILLRATE']
del test['PO_NUM']
del test['PO_ORD']
del test['PO_DUE']
del test['DATE_RECV']
del test['SHORTED']
del test['AMT_RECV']
print(test.dtypes)

BUS_GRP_NM          object
DIVISION            object
VEND_NUM            object
DST_CNTR            object
FACILITY            object
CORP_ITEM_CD        object
DAYS_LATE            int64
ORDERED              int64
MONTH               object
WEEK                object
DAY                 object
OrderFulfillment     int32
dtype: object
BUS_GRP_NM          object
DIVISION            object
VEND_NUM            object
DST_CNTR            object
FACILITY            object
CORP_ITEM_CD        object
DAYS_LATE            int64
ORDERED              int64
MONTH               object
WEEK                object
DAY                 object
OrderFulfillment     int32
dtype: object


In [71]:
# Seperate Target column from Train Data
TrainCols = list(train.columns.values)
Xtrain = train[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = train[['OrderFulfillment']].copy()
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
TestCols = list(test.columns.values)
Xtest = test[TrainCols[0:len(TrainCols)-1]].copy()
Ytest = test[['OrderFulfillment']].copy()
print("Test Set shape:")
print(Xtest.shape)
print(Ytest.shape)

Train Set shape:
(40112, 11)
(40112, 1)
Test Set shape:
(5000, 11)
(5000, 1)


In [72]:
#List of Categorical & Continous Features
CategoricalFeatures = ['BUS_GRP_NM','CORP_ITEM_CD','DIVISION','VEND_NUM','FACILITY','MONTH','WEEK','DAY']

ContinousFeatures = ['DAYS_LATE','ORDERED']

In [73]:
# OneHotEncoding (fit & transform)
# OneHotEncoding is to be done on Categorical variables after joining train and test data
pd.options.mode.chained_assignment = None
Xtrain['Dataset']=1
Xtest['Dataset']=0
combined=pd.concat([Xtrain,Xtest])
df=pd.get_dummies(combined[CategoricalFeatures])
combined=pd.concat([combined[ContinousFeatures],combined["Dataset"],df],axis=1)
#Seperating the train and test data after OneHotEncoding
Xtrain=combined[combined['Dataset']==1]
Xtest=combined[combined['Dataset']==0]
Xtrain.drop(['Dataset'],axis=1,inplace=True)
Xtest.drop(['Dataset'],axis=1,inplace=True)
print(Xtrain.shape)
print(Xtest.shape)


(40112, 788)
(5000, 788)


In [74]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt #To plot graphs

In [75]:
# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size = .30, random_state = 7, stratify=Ytrain)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(28078, 788) (28078, 1)
(12034, 788) (12034, 1)


In [77]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_train,Y_train)
clf_predict=clf.predict(X_test)
print("Accuracy Score (training data) for Decision Tree:{0:6f}".format(clf.score(X_train,Y_train)))
print("Accuracy Score (validation data) for Decision Tree:{0:6f}".format(clf.score(X_test,Y_test)))
print("AUC score on validation data:", metrics.roc_auc_score(Y_test,clf_predict))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(Y_test,clf_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Y_test, clf_predict))

Accuracy Score (training data) for Decision Tree:0.999822
Accuracy Score (validation data) for Decision Tree:0.705667
AUC score on validation data: 0.7056672760511884
Confusion Matrix for Decision Tree
[[4309 1708]
 [1834 4183]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

           0       0.70      0.72      0.71      6017
           1       0.71      0.70      0.70      6017

    accuracy                           0.71     12034
   macro avg       0.71      0.71      0.71     12034
weighted avg       0.71      0.71      0.71     12034



In [78]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
nb = GaussianNB()
# Train the model using the training sets
nb.fit(X_train,Y_train)
#Predict Output
nb_predict=nb.predict(X_test)
#predicted= model.predict([[0,2]]) # 0:Overcast, 2:Mild
print("Accuracy Score (training data) for Naive Bayes:{0:6f}".format(nb.score(X_train,Y_train)))
print("Accuracy Score (validation data) for Naive Bayes:{0:6f}".format(nb.score(X_test,Y_test)))
print("AUC score on validation data:", metrics.roc_auc_score(Y_test,nb_predict))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(Y_test,nb_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Y_test, nb_predict))

Accuracy Score (training data) for Naive Bayes:0.689864
Accuracy Score (validation data) for Naive Bayes:0.684560
AUC score on validation data: 0.6845604121655311
Confusion Matrix for Decision Tree
[[5089  928]
 [2868 3149]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

           0       0.64      0.85      0.73      6017
           1       0.77      0.52      0.62      6017

    accuracy                           0.68     12034
   macro avg       0.71      0.68      0.68     12034
weighted avg       0.71      0.68      0.68     12034



In [48]:
#Save predictions on actual test data
clf_testpred = gbc_hp.predict(Xtest)
df_DT=pd.DataFrame()
df_DT= finaldf
df_DT['OrderFulfillment_PredictFlag']= clf_testpred
#df_DT.reset_index(drop=True, inplace=True)
print (df_DT.head(3))
export_csv = df_DT.to_csv(r'C:\D_Drive\Data\VtoDC_Test.csv',index = False)


   BUS_GRP_NM DIVISION VEND_NUM DST_CNTR FACILITY CORP_ITEM_CD  DAYS_LATE  \
4      BAKERY       19        1     WPOA     5316     95100794          0   
7      BAKERY       25        1     WNCA     2533     95100794          0   
8      BAKERY       25        1     WNCA     2533     95100794          0   
23     BAKERY       30        1     WSLC     5217     95100794          3   
24     BAKERY        5        1     WDEN      512     95100796          1   

    ORDERED MONTH WEEK DAY  OrderFulfillment  OrderFulfillment_PredictFlag  
4       300    12   50  15                 0                             1  
7       930    12   48   2                 0                             1  
8       930    12   49   9                 0                             1  
23      300    12   48   2                 0                             1  
24      510    12   50  14                 1                             1  
